# Probabilistic Shape Biases

### This Notebook is a complement to the paper

#### **Probabilistic Estimators of Lagrangian Shape Biases: Universal Relations and Physical Insights** [(Maion, Stucker, Angulo, 2024)](https://arxiv.org/pdf/2409.14995)

The calculations necessary to reach equations (29,30) and (42,43) of that paper can be quite cumbersome. Therefore, we have developed a semi-numerical approach to perform those calculations.

In [3]:
import sympy
import numpy as np
import os
from sympy import *

os.chdir("/cosmos_storage/home/fgmaion/IA-probabilistic-bias")

import IA_isotropic_tensors as IA_it
import isotropic_tensors as it

%load_ext autoreload

In [4]:
IA_its = IA_it.IsotropicTensorSystem(usecache=True)
D_its  = it.IsotropicTensorSystem(usecache=True)

## Bias Estimators

From equation (26) of Maion et. al (2024) we know that the probability distribution for the tidal-field is given by:
$$
p(\mathbf{T}) = N \exp\left[-\frac{1}{2}\mathbf{T}\mathbf{C}_T^\dagger\mathbf{T} \right],
$$
where the pseudo-inverse covariance is obtained in Stucker et. al (2023), and shown to be
$$
\mathbf{C}_T^\dagger = \frac{1}{\sigma^2}\mathbf{J}_{22} + \frac{15}{2\sigma^2}\mathbf{J}_{2=2},
$$
in which $\mathbf{J}$ are the orthogonal isotropic tensors described in these two papers; we also refer the reader to another notebook in this repository <code>tensor_algebra.ipynb</code>, where the algebraic properties of these tensors and their definitions are explored in more depth.

The purpose of this Notebook is to use semi-numeric methods defined in <code>IA_isotropic_tensors.py</code> to explicitly compute the expressions for the bias estimators derived in Maion et al (2024):
$$
c_{J_{22}} = \left\langle \mathbf{C}_T^\dagger \mathbf{T}\otimes\mathbf{I}\overset{(4)}{\cdot} \frac{\mathbf{J}_{22}}{||\mathbf{J}_{22}||^2}_g \right\rangle\\
c_{J_{2=2}} = \left\langle \mathbf{C}_T^\dagger \mathbf{T}\otimes\mathbf{I}\overset{(4)}{\cdot} \frac{\mathbf{J}_{2=2}}{||\mathbf{J}_{2=2}||^2}_g \right\rangle
$$
and
$$
c_{J_{2=22}} = \left\langle \frac{1}{p(\mathbf{T})} \left( p\frac{\partial F}{\partial \mathbf{T}}\frac{\partial F}{\partial \mathbf{T}}-p \frac{\partial^2 F}{\partial \mathbf{T}^2} + 2 \frac{\partial p}{\partial \mathbf{T}} \frac{\partial F}{\partial \mathbf{T}} + \frac{\partial^2 p}{\partial \mathbf{T}^2} \right) \otimes \mathbf{I} \overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2}  \right\rangle_{\mathrm{g}}\\
c_{J_{2-2-2-}} = \left\langle \frac{1}{p(\mathbf{T})} \left( p\frac{\partial F}{\partial \mathbf{T}}\frac{\partial F}{\partial \mathbf{T}}-p \frac{\partial^2 F}{\partial \mathbf{T}^2} + 2 \frac{\partial p}{\partial \mathbf{T}} \frac{\partial F}{\partial \mathbf{T}} + \frac{\partial^2 p}{\partial \mathbf{T}^2} \right) \otimes \mathbf{I} \overset{(6)}{\cdot}\frac{\mathbf{J}_{2-2-2-}}{||\mathbf{J}_{2-2-2-}||^2}  \right\rangle_{\mathrm{g}}.
$$

Let us start with the first two, which are a bit simpler, and then pass to the next two.

## $c_{J_{22}}, c_{J_{2=2}}$

The first step is to represent the pseudo-inverse covariance in semi-numeric form

In [7]:
potderiv = (2,)

Ci, sigma, As, Js = IA_its.pseudo_inverse_covariance_matrix(potderiv, full_output=True)

nderiv = len(potderiv)

#coefficients that can be combined to basis tensors to get full expression of C_dagger
Ci_coeffrep = []
for i in range(0,nderiv):
    Ci_coeffrep.append([])
    for j in range(0,nderiv):
        Ci_coeffrep[i].append((As[i][j], (potderiv[i], potderiv[j])))

#fully numerical version of pseudo-inverse covariance
C_dag = As[0][0][0]*IA_its.obasis["22"]["J22"].N() + As[0][0][1]*IA_its.obasis["22"]["J2=2"].N()

#print symbolic expression of this matrix
basis = IA_its.obasis['22']
keys = list(basis.keys())
symbol_res = np.sum( basis[key].symbol() * Ci_coeffrep[0][0][0][i] for i, key in enumerate(keys)  )
symbol_res

/tmp/ipykernel_260344/2799662498.py:20: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  symbol_res = np.sum( basis[key].symbol() * Ci_coeffrep[0][0][0][i] for i, key in enumerate(keys)  )


J_22/sigma0**2 + 15*J_2=2/(2*sigma0**2)

and the next step is to use functions of the code to perform the relevant tensor operations

In [8]:
res = 0
I = IA_its.I[0]

for i in range(0, nderiv):
    crep = IA_its.coeffrep_multidot(IA_its.obasis["22"]["J2=2"], Ci_coeffrep[0][i], products=(1,))

    res += IA_its.coeffrep_to_symbol(*crep) * IA_its.x[i] * I / IA_its.obasis["22"]["J2=2"].norm2()

sympy.together(res)

3*J_2=2*T*I/(2*sigma0**2)

There is also a pre-defined function that incorporates these steps into one single function call. The parameters that have to be provided are simply to term to be computed, and up to which order in the derivatives of the potential you wish to go (so far this is only implemented for the case <code>potderiv=(2,)</code> )

### $c_{J_{22}}$

In [9]:
IA_its.IA_symbol_bias_deriv1(term="J22", potderiv=(2,))

J_22*T*I/(3*sigma0**2)

### $c_{J_{2=2}}$

In [10]:
IA_its.IA_symbol_bias_deriv1(term="J2=2", potderiv=(2,))

3*J_2=2*T*I/(2*sigma0**2)

## Now let's move to the calculation of the estimators for the higher-order parameters

For that we need the second-derivative of the probability distribution
$$
\frac{1}{p}\frac{\partial^2 p}{\partial \mathbf{T}^2} = \frac{1}{4}\left[ \mathbf{C}_T^\dagger(\mathbf{T}+\mathbf{T}^T) \right] \otimes \left[ \mathbf{C}_T^\dagger(\mathbf{T}+\mathbf{T}^T)\right] - \mathbf{C}_T^\dagger,
$$
and we will need the derivatives of $F$, the large-scale bias function, evaluated at $\mathbf{T}=0$
$$
\frac{\partial F}{\partial \mathbf{T}}(0) = \mathbf{B}_{T,1}
$$
$$
\frac{\partial^2 F}{\partial \mathbf{T}^2}(0) = \mathbf{B}_{T,2},
$$
in which $\mathbf{B}_{T,1}$ and $\mathbf{B}_{T,2}$ are the first and second-order generalized density-bias tensors
$$
\mathbf{B}_{T,1} = b_1\mathbf{J}_{2}
$$
$$
\mathbf{B}_{T,2} = b_2\mathbf{J}_{22} + 2b_{K^2}\mathbf{J}_{2=2}.
$$

Let us start with the contributions that come from the second derivatives of the probability distribution, as those will be the main ones. All we need to do then is to contract this second derivative with the relevant tensor, and normalize it by its amplitude.

## $c_{J_{2-2-2-}}$

In [11]:
res=0
#norm of the relevant tensor
norm2 = sympy.Rational(IA_its.obasis["222"]["J2-2-2-"].norm2()).limit_denominator(10**8)

for i in range(0, nderiv):
    for j in range(0, nderiv):
        #2-dimensional dot product of pseudo-inverse covariance matrices with basis tensor
        crep = IA_its.coeffrep_multidot(IA_its.obasis["222"]["J2-2-2-"], Ci_coeffrep[0][i], Ci_coeffrep[0][j], products=(1,1))
        #outer product of the result to the two instances of the tidal fiend and the moment of inertia, I
        res += IA_its.coeffrep_to_symbol(*crep) * IA_its.x[i] * IA_its.x[j] * IA_its.I[0] / norm2

# two-dimensional dot product of pseudo-inverse covariance matrices with basis tensor
crep = IA_its.coeffrep_multidot(IA_its.obasis["222"]["J2-2-2-"], Ci_coeffrep[0][0], products=(2,))
#
res += - IA_its.coeffrep_to_symbol(*crep) * IA_its.I[0] / norm2
res

135*J_2-2-2-*T**2*I/(7*sigma0**4)

There is also a ready function to do that for you

In [12]:
IA_its.IA_symbol_bias_deriv2(term="J2-2-2-", potderiv=(2,))

135*J_2-2-2-*T**2*I/(7*sigma0**4)

## $c_{J_{22=2}}$

In [13]:
IA_its.IA_symbol_bias_deriv2(term="J22=2", potderiv=(2,))

3*J_222=*T**2*I/(2*sigma0**4)

### Now we need to go back and compute the contributions from the derivatives of the number-density bias function

I will demonstrate how to perform this calculation for $c_{J2=22}$, since that one receives a non-zero calculation. The same is completely analogous for $c_{J_{2-2-2-}}$

Remembering the expression for this coefficient, we have
$$
c_{J_{2=22}} = \left\langle \frac{1}{p(\mathbf{T})} \left( p\frac{\partial F}{\partial \mathbf{T}}\frac{\partial F}{\partial \mathbf{T}}-p \frac{\partial^2 F}{\partial \mathbf{T}^2} + 2 \frac{\partial p}{\partial \mathbf{T}} \frac{\partial F}{\partial \mathbf{T}} + \frac{\partial^2 p}{\partial \mathbf{T}^2} \right) \otimes \mathbf{I} \overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2}  \right\rangle_{\mathrm{g}}\\
$$
so we need to compute
$$
\frac{\partial^2 F}{\partial \mathbf{T}^2}(0) \otimes \mathbf{I}\overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2} = \mathbf{B}_{T,2} \otimes \mathbf{I}\overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2}\\ 
$$
and
$$
2 \frac{\partial p}{\partial \mathbf{T}}\otimes\frac{\partial F}{\partial \mathbf{T}}(0) \otimes \mathbf{I}\overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2} = -2\mathbf{C}_T^\dagger \mathbf{T} \otimes \mathbf{B}_{T,1} \otimes \mathbf{I}\overset{(6)}{\cdot}\frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2}\\
$$

It is not necesary to compute the contribution of
$$
\frac{\partial F}{\partial \mathbf{T}}(0)\frac{\partial F}{\partial \mathbf{T}}(0)\otimes \mathbf{I} \overset{(6)}{\cdot} \frac{\mathbf{J}_{2=22}}{||\mathbf{J}_{2=22}||^2}
$$
since this will clearly not have a trace-free contribution, and will return a null contribution

### Define the density bias tensors

In [14]:
b1, b2, b3, bK2, bdk, bkk = sympy.symbols('b1 b2 b3 bK2 bdk bkk')

In [16]:
# Generalized bias tensors
B_1 = b1*D_its.obasis['2']['J2'].N()
B_2 = b2*D_its.obasis['22']['J22'].N() + 2*bK2*D_its.obasis['22']['J2=2'].N()
B_3 = b3*D_its.obasis['222']['J222'].N() + bdk*D_its.obasis['222']['J2=22'].N() + bkk*D_its.obasis['222']['J2-2-2-'].N()

$\mathbf{J}_{2=22} \overset{(4)}{\cdot} \left( \mathbf{C}_T^\dagger \otimes \mathbf{B}_1 \right) = $

In [17]:
res = IA_its.decompose(np.einsum('abijmn,ijkl,mn', IA_its.obasis["222"]["J2=22"].N(), C_dag, B_1, optimize='optimal'), "22")
basis = IA_its.obasis['22']
keys = list(basis.keys())
symbol_res = np.sum( basis[key].symbol() * res[i] for i, key in enumerate(keys)  )
symbol_res

/tmp/ipykernel_260344/1695721579.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  symbol_res = np.sum( basis[key].symbol() * res[i] for i, key in enumerate(keys)  )


45*b1*J_2=2/(2*sigma0**2)

In [19]:
IA_its.obasis["222"]["J2=22"].norm2()

15

and these results should now be plugged into the equation above, performing the additional 4-dimensional dot product with $\mathbf{T}$ and $\mathbf{I}$

$-2\frac{1}{|J_{2=22}|^2}\frac{45 b_{1} J_{2=2}}{2 \sigma_{0}^{2}} I T = -2b_1\frac{3}{2\sigma_0^2}\mathrm{Tr}(KI) = -2b_1c_K $